# Pulling Spotify Data

Import libraries

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import psycopg2
import pandas as pd
import sqlalchemy as sql
from requests.exceptions import HTTPError

Create a database called "spotify" in postgres, then enter postgres database credentials

In [ ]:
password="enter_password"
db_name="spotify"

Create engine to interact with postgres database

In [ ]:
engine = sql.create_engine('postgresql+psycopg2://postgres:'+password+'localhost:5432/'+db_name)

Create "spotify_songs" table

In [ ]:
query='''

CREATE TABLE spotify_songs(

track_id TEXT,
artist TEXT,
song_name TEXT,
popularity REAL,
album TEXT,
isrc TEXT,
danceability REAL,
energy REAL,
key REAL,
loudness REAL,
mode REAL,
speechiness REAL,
acousticness REAL,
instrumentalness REAL,
liveness REAL,
valence REAL,
tempo REAL,
type TEXT,
id TEXT,
uri TEXT,
track_href TEXT,
analysis_url TEXT,
duration_ms TEXT,
time_signature TEXT
)

'''

engine.exectute(query)

Create connection to Spotify API

- Log in to your Spotify account at: https://developer.spotify.com/dashboard/login
- Create an app
- After creating app you will receive a Client ID and Client Secret
- You need to either set an environmental variable SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET, or feed them as parameters into SpotifyOAUTH (parameters being client_id and client_secret)
- Within your Spotify developer dashboard go to your app and provide a redirect uri. This must match the redirect_uri that is a parameter of SpotifyOAUTH
- You also have to provide a scope. Read more about scopes here: https://developer.spotify.com/documentation/general/guides/scopes/

In [ ]:
scope = "user-library-read"
redirect_uri = 'http://localhost:8080'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(redirect_uri=redirect_uri,scope=scope))

The below are functions used to pull the song data and push it to Postgres. The methodology here is to get all of Spotify;s major categories, find all playlists from those categories and then get all song and info from those playlists. Turn it into a dataframe and push it to Postgres.

In [ ]:
#Get all Spotify categories
def get_categories(sp):
    all_categories=[]
    for i in sp.categories()['categories']['items']:
        all_categories.append(i['id'])
    return all_categories

#Given a list of categories, find the playlists within that category
def get_playlists(sp, all_categories):
    playlist=[]

    for cat in all_categories:
        try:
            for i in sp.category_playlists(category_id=cat)['playlists']['items']:
                playlist.append(i['id'])
        except:
            continue
    return playlist

#Given a list of playlists, find all songs for each playlist and extract info about those songs. 
#Provide a list of column names and return a dataframe of the data
def get_song_info(sp,playlist,columns):
    song_info =[]
    for index,play in enumerate(playlist):
        print("Working on Song:",index)
        try:
            for i in sp.playlist(playlist_id=play)['tracks']['items']:
                all_feat = []
                track_id=i['track']['id']
                all_feat.append(track_id)
                audio_feat = sp.audio_features(track_id)
                artist = i['track']['album']['artists'][0]['name']
                all_feat.append(artist)
                song_name = i['track']['name']
                all_feat.append(song_name)
                popularity = i['track']['popularity']
                all_feat.append(popularity)
                album = i['track']['album']['name']
                all_feat.append(album)
                isrc = i['track']['external_ids']['isrc']
                all_feat.append(isrc)
                for i,val in audio_feat[0].items():
                    all_feat.append(val)
                song_info.append(all_feat)
        except:
            continue        
    df=pd.DataFrame(song_info,columns=columns)
    return df

#Push dataframe to your Postgres Database
def push_to_db(df):
    df.to_sql('spotify_songs', engine, schema='public', if_exists='append', index=False)
    return

Set column values for dataframe (same as values used to create spotify_songs table):

In [ ]:
columns = ['track_id','artist','song_name','popularity','album','isrc','danceability',
'energy',
'key',
'loudness',
'mode',
'speechiness',
'acousticness',
'instrumentalness',
'liveness',
'valence',
'tempo',
'type',
'id',
'uri',
'track_href',
'analysis_url',
'duration_ms',
'time_signature']

Pull the songs and push the dataframe to Postgres:

In [ ]:
categories=get_categories(sp)
playlists=get_playlists(sp,categories)
df=get_song_info(sp,playlists,columns)
df=df.dropDuplicates()
push_to_db(df)